In [ ]:
import numpy as np
import pandas as pd
import os
        
os.chdir('/kaggle/input/home-credit-default-risk')

In [ ]:
train = pd.read_csv('application_train.csv')
train.head()

In [ ]:
train_sample = train.sample(1000)
train_sample.info()

In [ ]:
train_sample.select_dtypes('object').head()

In [ ]:
binary = lambda x, val='Y': 1 if x == val else 0

#def apply_binary(**kwargs):
#    for column, val in kwargs.items():
#        train_sample[column] = train_sample[column].apply(binary, val=val)

train_sample['CODE_GENDER']     = train_sample['CODE_GENDER'].apply(binary, val='F')
train_sample['FLAG_OWN_CAR']    = train_sample['FLAG_OWN_CAR'].apply(binary)
train_sample['FLAG_OWN_REALTY'] = train_sample['FLAG_OWN_REALTY'].apply(binary)
train_sample['NAME_CONTRACT_TYPE'] = train_sample['NAME_CONTRACT_TYPE'].apply(binary, val='Cash loans')

#train_sample.rename({'CODE_GENDER':'FEMALE', 'NAME_CONTRACT_TYPE':'Cash_or_revolving'})
#train_sample.columns

In [ ]:
train_sample.select_dtypes('object').nunique()

In [ ]:
train_sample.NAME_CONTRACT_TYPE.unique()

In [ ]:
train_sample.EMERGENCYSTATE_MODE.unique()
train_sample.EMERGENCYSTATE_MODE.isnull().sum()

In [ ]:
docs_cols = [col for col in train_sample.columns if 'FLAG_DOCUMENT' in col]
docs = train_sample[docs_cols]
docs.head()

In [ ]:
docs['TOTAL'] = docs.sum(axis=1)
docs['TOTAL'].describe()

In [ ]:
docs.sum()

In [ ]:
docs_to_remove = [col for col in docs.columns if docs[col].sum() < 15]
train_sample = train_sample.drop(docs_to_remove, axis=1)
train_sample.select_dtypes('int').columns

In [ ]:
train_sample.select_dtypes('int').head()

In [ ]:
train_sample.select_dtypes('float').head()

In [ ]:
def null_counts(df):
    
    null_counts = dict(df.isnull().sum())
    
    for key, value in null_counts.items():
        if value != 0:
            print(key, value)
                  

In [ ]:
train_sample.dropna(axis=0).shape
#only 29 full rows which is smaller than ideal, so will have to impute some

In [ ]:
train_sample.corr()['TARGET'].sort_values()

In [ ]:
train_sample = train.drop('FLAG_MOBIL', axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
objects = train_sample.select_dtypes('object')
objects.isnull().sum()

In [ ]:
drop_cols = ['OCCUPATION_TYPE', 'NAME_TYPE_SUITE', 'FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE']
train_sample = train_sample.dropna(subset=drop_cols, axis=0, how='any')
objects = train_sample[objects.columns]
train_sample.isnull().sum()

In [ ]:
train_sample = train_sample.drop(objects.columns, axis=1)

In [ ]:
for col in train_sample.columns:
    mean = train_sample[col].mean()
    train_sample[col].fillna(mean, inplace=True)
train_sample.isnull().sum()

In [ ]:
train_sample = train_sample.join(objects, how='outer')

In [ ]:
dummies = OneHotEncoder(drop='first')
x = dummies.fit_transform(train_sample).toarray()

In [ ]:
x.shape

In [ ]:
train_sample.data = x
#train_sample.isnull().sum()
train_sample.dtypes

In [ ]:
reg = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(train_sample.drop('TARGET', axis=1), train_sample['TARGET'], test_size=.2, random_state=11)

In [ ]:
reg.fit(X_train, y_train)
reg